In [ ]:
pip install --upgrade pip
pip install langchain
# pip install openai
pip install pandas
pip install tabulate
pip install transformers
# pip install llama-cpp-python
pip install llama-cpp-python==0.1.48
# install agent tools
pip install google-search-results
pip install wikipedia
# playwright tool
pip install playwright
pip install beautifulsoup4

In [ ]:
import requests
import os
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms.base import LLM
from typing import Optional, List
import re

In [ ]:
if(False): # run the following code to download a model like wizardLM-7B.ggml.q4_0.bin from huggingface.co
  l7b="https://huggingface.co/hlhr202/llama-7B-ggml-int4/resolve/main/ggml-model-q4_0.bin"
#   l13b="https://huggingface.co/Drararara/llama-13B-ggml/resolve/main/ggml-model-q4_0.bin"   
#   a7b="https://huggingface.co/hlhr202/alpaca-7B-ggml-int4/resolve/main/ggml-alpaca-7b-q4.bin"
#   a13b="https://huggingface.co/Pi3141/alpaca-lora-13B-ggml/resolve/main/ggml-model-q4_0.bin"  
#   o13b="https://huggingface.co/Black-Engineer/oasst-llama13b-ggml-q4/resolve/main/qunt4_0.bin" 
#   g4a="https://huggingface.co/eachadea/ggml-gpt4all-7b-4bit/resolve/main/gpt4all-lora-quantized-ggml.bin"
#   k7b="https://huggingface.co/TheBloke/koala-7B-GPTQ-4bit-128g-GGML/resolve/main/koala-7B-4bit-128g.GGML.bin" 
#   v7b="https://huggingface.co/eachadea/ggml-vicuna-7b-1.1/resolve/main/ggml-vicuna-7b-1.1-q4_0.bin"
#   v13b="https://huggingface.co/eachadea/ggml-vicuna-13b-1.1/resolve/main/ggml-vicuna-13b-1.1-q4_0.bin"
#   wizVic="https://huggingface.co/TheBloke/wizard-vicuna-13B-GGML/resolve/main/wizard-vicuna-13B.ggml.q4_0.bin" 
#   gpt4Vicuna="https://huggingface.co/TheBloke/gpt4-x-vicuna-13B-GGML/resolve/main/gpt4-x-vicuna-13B.ggml.q4_0.bin"
  
  weights=requests.get(l7b)
  with open("weights.bin","wb") as out_file:
    out_file.write(weights.content)

In [ ]:
from llama_cpp import Llama
llamallm = Llama(model_path="D:\Workspace\LangChain\ggml-model-q4_0.bin",n_ctx=2048)
output = llamallm("Software developer", echo=True)
print(output)

In [ ]:
class CustomLLM(LLM):  
  def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:    
    print("***\n"+prompt+"\n***")
    output = llamallm(prompt, echo=False) #, stop=["Q:", "\n"], max_tokens=100,     
    output = output["choices"][0]["text"]
    output = re.sub("\nAction:(.*)[Dd]atabase(.*)","\nAction: Database",output)    
    output = re.sub("\nAction:(.*)Wikipedia(.*)","\nAction: Wikipedia",output)
    if(output.find("\nAction:")>=0 and output.find("\nObservation:")>output.find("\nAction:")): return(output[0:output.find("\nObservation:")])
    else: return(output)
  @property
  def _llm_type(self) -> str:
    return "custom"

llm=CustomLLM()

In [ ]:
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,
)

async_browser = create_async_playwright_browser()
playwrighttoolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
playwrighttools = playwrighttoolkit.get_tools()

os.environ["SERPAPI_API_KEY"] = ""
tools = load_tools(["requests_get", "serpapi", "python_repl", "wikipedia"], llm=llm)

In [ ]:
agent = initialize_agent(playwrighttools + tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("Write the code for index website page like https://langchain.com/")